## Import all the necessary library

In [1]:
import pandas as pd
import numpy as np

<div class="alert alert-info" role="alert">
  <span style="font-size: larger;"><strong>1. Read all the csv files into DataFrames</strong></span>
</div>

In [2]:
df_customers = pd.read_csv("cleaned_data\customers.csv")
df_shirts = pd.read_csv("cleaned_data\shirts.csv")
df_invoice_details = pd.read_csv("cleaned_data\invoice_details.csv")
df_invoices = pd.read_csv("cleaned_data\invoices.csv")
df_demands = pd.read_csv("cleaned_data\demands.csv")
df_imports = pd.read_csv("cleaned_data\imports.csv")
df_shops = pd.read_csv("cleaned_data\shops.csv")
df_import_details = pd.read_csv("cleaned_data\import_details.csv")

<div class="alert alert-info" role="alert">
  <span style="font-size: larger;"><strong>2. Calculate the percentage distribution of each category of shirt</strong></span>
</div>

In [3]:
# merge df_invoices and df_invoice_details and group by ShirtID for each unique shirt color and size
df = pd.merge(df_invoices, df_invoice_details, on="InvoiceID")
df = df.groupby('ShirtID').agg(Count=("Quantity", "sum")).reset_index()

# store necessary data into necessary variable
shirt_sold_each_category = df["Count"]
total_shirt_sold = df["Count"].sum()

# calculate the percentages of each category
percentage_for_each_category = shirt_sold_each_category/total_shirt_sold
percentage_for_each_category

0     0.018616
1     0.021682
2     0.022512
3     0.023342
4     0.023470
5     0.022895
6     0.022480
7     0.023374
8     0.022576
9     0.023118
10    0.022033
11    0.023502
12    0.022576
13    0.026216
14    0.022097
15    0.020372
16    0.020979
17    0.023150
18    0.018616
19    0.024108
20    0.024364
21    0.022991
22    0.021522
23    0.023917
24    0.022544
25    0.022480
26    0.020021
27    0.023278
28    0.019702
29    0.024396
30    0.019670
31    0.020851
32    0.023661
33    0.020756
34    0.021490
35    0.022129
36    0.023629
37    0.020532
38    0.021809
39    0.021011
40    0.021139
41    0.023757
42    0.020724
43    0.022480
44    0.023438
Name: Count, dtype: float64

<div class="alert alert-info" role="alert">
  <span style="font-size: larger;"><strong>3. Calculate the recommeded stocking quantity by using the safety stock formula</strong></span>
</div>

In [4]:
# store lead_times and sales into varible, the lead_times are divided by 30 due to fact it's in term of months in our case
lead_times = (df_imports["AcutalWaitingDays"] - df_imports["ExpectedWaitingDays"])/30
number_of_month_to_stock_for = 6
sales = df_demands["QuantitySold"]

# using the safety stock formula and normal stock
safety_stock = ((np.max(lead_times) * np.max(sales)) - (np.average(lead_times) * np.average(sales))) * number_of_month_to_stock_for
stock = np.average(sales) * number_of_month_to_stock_for
stock = np.round(stock)

# apply the percentage distribution percentages of each category over the amount from the safety stock
quantity_for_each_category = percentage_for_each_category * stock
backup_quantity_for_each_category = percentage_for_each_category * safety_stock
print("Amount to stock:\n", list(quantity_for_each_category))
print("Amount to stock for backup:\n", list(backup_quantity_for_each_category))

Amount to stock:
 [80.70073761854584, 93.98936679758597, 97.58837053357603, 101.18737426956605, 101.74106715202606, 99.24944918095602, 97.44994731296102, 101.32579749018105, 97.86521697480602, 100.21841172526105, 95.51202222435099, 101.87949037264106, 97.86521697480602, 113.64546412491619, 95.78886866558099, 88.31401475237092, 90.94405594405595, 100.35683494587605, 80.70073761854584, 104.50953156432608, 105.61691732924609, 99.66471884280104, 93.29725069451096, 103.67899224063608, 97.72679375419101, 97.44994731296102, 86.7913593256059, 100.91052782833604, 85.4071271194559, 105.7553405498611, 85.26870389884088, 90.39036306159593, 102.57160647571607, 89.97509339975093, 93.15882747389597, 95.927291886196, 102.43318325510106, 89.00613085544592, 94.54305968004599, 91.08247916467094, 91.63617204713096, 102.98687613756107, 89.83667017913594, 97.44994731296102, 101.60264393141105]
Amount to stock for backup:
 [10.385977051867464, 12.096189396600355, 12.559371906632181, 13.022554416664006, 13.09

<div class="alert alert-info" role="alert">
  <span style="font-size: larger;"><strong>4. Function to calculate the quantity for each category and round the result</strong></span>
</div>

In [5]:
def round_array_to_interger(array):
    # store the orginal quantity_for_each_category into a numpy array and the original sum of the array
    original_array = array.copy()
    original_sum = np.sum(original_array)

    # find the fractional part of each number in the numpy array
    fractional_parts = original_array - np.floor(original_array)

    # round the array and find the sum of the rounded array
    rounded_array = np.round(original_array).astype(int)
    rounded_sum = np.sum(rounded_array)

    # calculate the difference between orginal sum and the sum from the rounded array
    difference = original_sum - rounded_sum

    # if difference is bigger than 0 distribution the difference to 
    # the biggest numbers that had been rounded down
    if difference > 0:
        sorted_indices = np.argsort(fractional_parts)[::-1]
        for idx in sorted_indices:
            if fractional_parts[idx] < 0.5:
                rounded_array[idx] += 1
                difference -= 1
                if difference == 0:
                    break
                    
    # if diffrence is smaller than 0 distribution the difference to
    # the smallest numbers that had been rounded up
    elif difference < 0:
        sorted_indices = np.argsort(fractional_parts)
        for idx in sorted_indices:
            if fractional_parts[idx] >= 0.5:
                rounded_array[idx] -= 1
                difference += 1
                if difference == 0:
                    break
                    
    return rounded_array

In [6]:
recommned_quantity_to_import = round_array_to_interger(quantity_for_each_category)
backup_quantity_to_import = round_array_to_interger(backup_quantity_for_each_category)
print("Recommended Quantity To Import", list(recommned_quantity_to_import))
print("Backup_Quantity_Import: ", list(backup_quantity_to_import))

Recommended Quantity To Import [81, 94, 98, 101, 102, 99, 97, 101, 98, 100, 95, 102, 98, 114, 96, 88, 91, 100, 81, 104, 106, 100, 93, 104, 98, 97, 87, 101, 85, 106, 85, 90, 103, 90, 93, 96, 102, 89, 95, 91, 92, 103, 90, 97, 102]
Backup_Quantity_Import:  [10, 12, 12, 13, 13, 12, 12, 13, 12, 12, 12, 13, 12, 14, 12, 11, 11, 12, 10, 13, 13, 12, 12, 13, 12, 12, 11, 12, 10, 13, 10, 11, 13, 11, 11, 12, 13, 11, 12, 11, 11, 13, 11, 12, 13]


<div class="alert alert-info" role="alert">
  <span style="font-size: larger;"><strong>5. Merge other attributes of each ShirtID with the Recommended_Quantity_To_Import and Optinal_Backup_Quantity_To_Import</strong></span>
</div>

In [7]:
# create a dictionary storing ShirtID and the Recommended_Quantity_To_Import for each ShirtID
data_recommended_quantity = {
    "ShirtID" : [i+1 for i in range(45)],
    "Recommended_Quantity_To_Import" : recommned_quantity_to_import,
    "Optinal_Backup_Quantity_To_Import" : backup_quantity_to_import
}

# create the DataFrame from the dictionary
df_recommended_quantity = pd.DataFrame(data_recommended_quantity)

# merge the df_shirts and df_recommended_quantity so that we get the ShirtID, other attributes
# and Recommended_Quantity_To_Import for each ShirtID in one DataFrame
df = pd.merge(df_shirts, df_recommended_quantity, on="ShirtID")
df = df.drop("StockQty", axis=1)
df.sort_values(by=["Color", "Size"], inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv("RecommendedStockingQuantity.csv", index=False)
df

,ShirtID,Color,Size,UnitPrice,Discount,ShopID,Recommended_Quantity_To_Import,Optinal_Backup_Quantity_To_Import
0,27,black,L,2,0.3,1,87,11
1,9,black,S,3,0.4,4,98,12
2,45,black,XL,9,0.2,3,102,13
3,38,black,XXL,9,0.4,2,89,11
4,35,blue,L,7,0.5,1,93,11
5,21,blue,M,7,0.4,1,106,13
6,2,blue,S,3,0.4,1,94,12
7,42,blue,XXL,6,0.1,2,103,13
8,39,brown,L,7,0.5,5,95,12
9,33,brown,S,5,0.3,4,103,13
